<a href="https://colab.research.google.com/github/sipocz/Twitter-Sentiment-Analysis-AI-Challenge/blob/main/Sentiment_Net.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [453]:
import pandas as pd

In [454]:
#emoji_url="https://github.com/sipocz/Twitter-Sentiment-Analysis-AI-Challenge/raw/5e7d7d29f58be438f33fa0c0bc2fd251dea0b453/emoji.py"
#!rm emoji.py
#!wget $emoji_url
#import emoji

In [455]:
! pip install emoji
import emoji

     |████████████████████████████████| 168 kB 5.4 MB/s 
  Created wheel for emoji: filename=emoji-1.6.0-py3-none-any.whl size=168256 sha256=9104e04278af7195701cf969e1a860eea5e80c28d0df5acc709e7e3626671efb
  Stored in directory: /root/.cache/pip/wheels/f7/d7/74/c720aaf345a042b0c2d74361873258c5e8649b7f11b2ccce49
Successfully built emoji


In [456]:

emoji.demojize("hello 👍")

'hello :thumbs_up:'

##Szószedet

In [461]:
!rm SampleSubmission*
!rm TEST*
!rm TRAIN*
!rm *zip*
!wget https://github.com/sipocz/Twitter-Sentiment-Analysis-AI-Challenge/raw/9674d3c8ef760a8d2c5cc2e3af8a71faa93edf7d/twitter_sentiment_analysis_ai_challenge-dataset.zip
!unzip twitter_sentiment_analysis_ai_challenge-dataset.zip
df=pd.read_csv("TEST.csv")

--2021-10-09 22:10:54--  https://github.com/sipocz/Twitter-Sentiment-Analysis-AI-Challenge/raw/9674d3c8ef760a8d2c5cc2e3af8a71faa93edf7d/twitter_sentiment_analysis_ai_challenge-dataset.zip
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/sipocz/Twitter-Sentiment-Analysis-AI-Challenge/9674d3c8ef760a8d2c5cc2e3af8a71faa93edf7d/twitter_sentiment_analysis_ai_challenge-dataset.zip [following]
--2021-10-09 22:10:55--  https://raw.githubusercontent.com/sipocz/Twitter-Sentiment-Analysis-AI-Challenge/9674d3c8ef760a8d2c5cc2e3af8a71faa93edf7d/twitter_sentiment_analysis_ai_challenge-dataset.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awa

In [462]:
df.head()
alltext1=df.text
print (len(alltext1))

2640


In [463]:
df2=pd.read_csv("TRAIN.csv")
alltext2=df2.text
print (len(alltext2))

12000


In [464]:
alltext1=list(alltext1)
alltext2=list(alltext2)
alltext=alltext1+alltext2
print(f" {len(alltext)}")


 14640


In [465]:
!pip install pyspellchecker
from spellchecker import SpellChecker

spell = SpellChecker()
print(spell.correction("@frustrating"))

     |████████████████████████████████| 2.7 MB 5.3 MB/s 
frustrating


In [476]:
import re

def remove_URL(text):
    """Remove URLs from a text string"""
    return re.sub(r"http\S+", "", text)

def remove_emoji(string):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)

def is_number(inp:str):
    a=re.search("[\d]",inp)
    if a==None: 
        return(False)
    return(True)

def strip_all_entities(text):
    import emoji
    import re,string
    entity_prefixes = ['“','@','#',"!","?",":","_"]
    for separator in  string.punctuation:
        if separator not in entity_prefixes :
            text = text.replace(separator,' ')
    words = []
    for word in text.split():
        word = word.strip()
        if word:
            if word[0] not in ['“','@','#',"!","?"]:
                words.append(word)

    return ' '.join(words)

def query_sentence(index):
    print(f"orig: {alltext[index]}")
    spell=SpellChecker()
    inp=alltext[index]
    inp=remove_URL(inp)
    inp=emoji.demojize(inp)
    print("demojize: ",inp)
    inp=strip_all_entities(inp)
    print("STRIP: ",inp)
    txt=inp.replace(","," ").replace("."," ").replace("!"," _exclamation_ ").replace("#"," ").replace("???","?").replace("??","?").replace("?"," _questionmark_ ").replace("  "," ").replace("  "," ").replace('"',"").replace("“","")
    print("after replace=",txt)
    #print(txt)
    normal=""
    for i in txt.split(" "):
        w1=i.lower().strip()
        if "_" in w1 or ":" in w1:
            w2=w1
        else:
            if w1 !="":
                w2=spell.correction(w1).strip()
            else:
                w2=""
            if w2=="i":
                pass
                #print(f"chek: {w1}, {w2}")
            if is_number(w2):
                w2=""    
        normal += w2+" "
    normal=normal.replace("::",": :")
    print(f"  conv: {normal}")
    return(normal)

##Mentés

In [478]:
t=[]
for i in range(len(alltext)):
    print(f"{i}", end=" ")
    t.append(query_sentence(i))


A streamkimeneten csak az utolsó 5000 sor látható.
  conv: just sent thank u it please do your confirmation number if relight booking problems is needed thank you ey 
13643 orig: @JetBlue Not trying to make you look bad, on your website it says: "Due to weather in the Charleston, NC" Its actually in SC, not NC
demojize:  @JetBlue Not trying to make you look bad, on your website it says: "Due to weather in the Charleston, NC" Its actually in SC, not NC
STRIP:  Not trying to make you look bad on your website it says: Due to weather in the Charleston NC Its actually in SC not NC
after replace= Not trying to make you look bad on your website it says: Due to weather in the Charleston NC Its actually in SC not NC
  conv: not trying to make you look bad on your website it says: due to weather in the charleston no its actually in so not no 
13644 orig: @United super lame that you charge $8 for tv on an international flight to the US. It isn’t 1997!
demojize:  @United super lame that you charge

In [513]:
s=set()
for sent in t:
    wlist=sent.strip().split()
    for w in wlist:
        s.add(w)
print (len(s))
ordered=list(s)
ordered.sort()

9627


In [514]:
print(ordered)

['0_0', '10:00p', '10:00pm', '10:01', '10:01pm', '10:05am', '10:05pm', '10:07p', '10:10', '10:14am', '10:15am', '10:20', '10:20pm', '10:30', '10:30a', '10:30pm', '10:35', '10:39', '10:45', '10:45pm', '10:50am', '10:51', '10:51pm', '10:55', '10:55pm', '11:00', '11:00pm', '11:10p', '11:15', '11:15p', '11:20', '11:21p', '11:23', '11:30', '11:30a', '11:30am', '11:30pm', '11:31', '11:35', '11:40', '11:40am', '11:45am', '11:47', '11:50', '11:50pm', '11:51', '11:55am', '11:56', '11:59', '12:00', '12:08', '12:13pm', '12:15', '12:17pm', '12:20', '12:26pm', '12:30', '12:30am', '12:40', '15:30', '1:', '1:00pm', '1:02', '1:05', '1:10', '1:15', '1:23', '1:30', '1:30am', '1:30pm', '1:34', '1:36', '1:40pm', '1:45', '1:45pm', '1pm10:15', '2:', '2:00', '2:00pm', '2:10pm', '2:12', '2:14', '2:15', '2:20', '2:26', '2:30', '2:35pm', '2:45', '2:45hrs', '2:50pm', '2:55', '2:55pm', '2day:', '3:', '3:00am', '3:05pm', '3:15', '3:22', '3:30', '3:30am', '3:30pm', '3:35', '3:50', '3:55', '3:55pm', '3a:right_arrow:

In [515]:
sentence=pd.DataFrame(t)
sentence.columns=["Sentence"]
sentence.to_csv("Sentences_emoji.csv")

In [517]:
wordlist=pd.DataFrame(ordered)
wordlist.columns=["Words"]
wordlist.to_csv("words_emoji.csv")

###---------------------------------


###Adatbetöltés SZAVAK BETÖLTÉSE

In [1]:
import pandas as pd

In [2]:
#words_url="https://github.com/sipocz/Twitter-Sentiment-Analysis-AI-Challenge/raw/bc5a87c1da7a3fdcd674325ca8d1ef417075fbd0/words.csv"
words_url="https://github.com/sipocz/Twitter-Sentiment-Analysis-AI-Challenge/raw/5b6a8293d5c08614e7997e477df13d6044dfad01/words_emoji_2.csv"

In [3]:
!rm words* 
!wget $words_url

rm: cannot remove 'words*': No such file or directory
--2021-10-10 08:20:17--  https://github.com/sipocz/Twitter-Sentiment-Analysis-AI-Challenge/raw/5b6a8293d5c08614e7997e477df13d6044dfad01/words_emoji_2.csv
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/sipocz/Twitter-Sentiment-Analysis-AI-Challenge/5b6a8293d5c08614e7997e477df13d6044dfad01/words_emoji_2.csv [following]
--2021-10-10 08:20:17--  https://raw.githubusercontent.com/sipocz/Twitter-Sentiment-Analysis-AI-Challenge/5b6a8293d5c08614e7997e477df13d6044dfad01/words_emoji_2.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 123520 (121K) [

In [4]:
words=pd.read_csv("words_emoji_2.csv",index_col=0)
words.head()
words_list=list(words["Words"])

In [5]:
word_dict={v:i+1 for i,v in enumerate(words_list)}

### adatbetöltés mondatok betöltése

In [6]:
sentences_url="https://github.com/sipocz/Twitter-Sentiment-Analysis-AI-Challenge/raw/bc5a87c1da7a3fdcd674325ca8d1ef417075fbd0/Sentences.csv"
sentences_url="https://github.com/sipocz/Twitter-Sentiment-Analysis-AI-Challenge/raw/b9567d54968bba4d03ce8c3cac5ecdc4fe1c6031/Sentences_emoji.csv"
sentences_url="https://github.com/sipocz/Twitter-Sentiment-Analysis-AI-Challenge/raw/5b6a8293d5c08614e7997e477df13d6044dfad01/Sentences_emoji_2.csv"


In [7]:
!rm Sentences*
!wget $sentences_url
sentences=pd.read_csv("Sentences_emoji_2.csv",index_col=0)
sentences.head()
sentences_list=list(sentences["Sentence"])

rm: cannot remove 'Sentences*': No such file or directory
--2021-10-10 08:20:54--  https://github.com/sipocz/Twitter-Sentiment-Analysis-AI-Challenge/raw/5b6a8293d5c08614e7997e477df13d6044dfad01/Sentences_emoji_2.csv
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/sipocz/Twitter-Sentiment-Analysis-AI-Challenge/5b6a8293d5c08614e7997e477df13d6044dfad01/Sentences_emoji_2.csv [following]
--2021-10-10 08:20:54--  https://raw.githubusercontent.com/sipocz/Twitter-Sentiment-Analysis-AI-Challenge/5b6a8293d5c08614e7997e477df13d6044dfad01/Sentences_emoji_2.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length:

##Tanulás

In [8]:
!rm *zip*
!rm TEST*
!rm TRAIN*
!rm SampleSubmission*
!wget https://github.com/sipocz/Twitter-Sentiment-Analysis-AI-Challenge/raw/9674d3c8ef760a8d2c5cc2e3af8a71faa93edf7d/twitter_sentiment_analysis_ai_challenge-dataset.zip
!unzip twitter_sentiment_analysis_ai_challenge-dataset.zip


rm: cannot remove '*zip*': No such file or directory
rm: cannot remove 'TEST*': No such file or directory
rm: cannot remove 'TRAIN*': No such file or directory
rm: cannot remove 'SampleSubmission*': No such file or directory
--2021-10-10 08:21:00--  https://github.com/sipocz/Twitter-Sentiment-Analysis-AI-Challenge/raw/9674d3c8ef760a8d2c5cc2e3af8a71faa93edf7d/twitter_sentiment_analysis_ai_challenge-dataset.zip
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/sipocz/Twitter-Sentiment-Analysis-AI-Challenge/9674d3c8ef760a8d2c5cc2e3af8a71faa93edf7d/twitter_sentiment_analysis_ai_challenge-dataset.zip [following]
--2021-10-10 08:21:00--  https://raw.githubusercontent.com/sipocz/Twitter-Sentiment-Analysis-AI-Challenge/9674d3c8ef760a8d2c5cc2e3af8a71faa93edf7d/twitter_sentiment_analysis_ai_challenge-dataset.zip
Resolving raw.githubus

In [9]:
df=pd.read_csv("TRAIN.csv")

In [10]:
df.head(4)

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570018445695148033,negative,1.0,Customer Service Issue,0.6560,United,NaN,tpensari,NaN,0,@united your customer service is terrible! Sto...,NaN,2015-02-23 16:32:42 -0800,NaN,NaN
1,568830255282040832,neutral,1.0,NaN,NaN,Southwest,NaN,leavenodoubt131,NaN,0,@SouthwestAir I will dm you now,NaN,2015-02-20 09:51:15 -0800,NaN,Tijuana
2,569296356143050754,negative,1.0,longlines,0.3462,United,NaN,ScottyNeuman,NaN,1,@united congratulations united. Fail again. Sk...,"[39.85760174, -104.66669625]",2015-02-21 16:43:22 -0800,"Montana, God's country\n",Pacific Time (US & Canada)
3,569936640669581313,positive,1.0,NaN,NaN,United,NaN,koploperfan1992,NaN,0,@united look at this beauty 😉 dc 10 united air...,"[51.322819, 5.3576218]",2015-02-23 11:07:38 -0800,Winterfell,NaN


In [11]:
for i in range(len(df)):
    if "😊" in df.loc[i].text:
        print (f"{i:6}. {df.loc[i].airline_sentiment:9} -- {df.loc[i].retweet_count:5} --  {df.loc[i].text}")

     8. positive  --     0 --  @SouthwestAir a nice trip back home after a looong vaca 😊🌴 http://t.co/9TRvfnCedL
   752. positive  --     0 --  @united Thank you!! 😊
  1447. neutral   --     0 --  @JetBlue "Do what you love and you'll never work a day in your life." 😊
  1494. positive  --     0 --  @SouthwestAir Great job with the Passbook integration! It’ll really help streamline the commute to San Fran! 😊☕📲✈
  1781. positive  --     0 --  @united EWR agent Barbara was FABULOUS and an example of CUST. SERV. A pleasure talking to you😊 http://t.co/KMQuLY9g5E
  2740. positive  --     1 --  @JetBlue @Maddie_Flood Your airline sounds outstanding and your Twitter feed is clearly extremely useful. Keep up the great work 😊
  2914. positive  --     0 --  @USAirways the entire flight crew on flight 738 from BOS to PHL is doing a wonderful job and making this experience not totally suck. 😊
  3109. positive  --     0 --  @JetBlue be flying soon to NYC on your airline, of all airlines I've flown y

In [12]:
for i in range(len(df)):
    if df.loc[i].retweet_count>0:
        print (f"{i:6}. {df.loc[i].airline_sentiment:9} -- {df.loc[i].retweet_count:5} --  {df.loc[i].text}")

     2. negative  --     1 --  @united congratulations united. Fail again. Sky at a gate with a plane full of people waiting on 2. Weather gets worse now we can't leave.
    10. neutral   --     3 --  @southwestair has 22 employees on Social Care Team, 4 staff the Listening Ctr. at any time-rest of spots filled by other areas. #RaganDisney
    66. negative  --     1 --  @united that may be true however after 4 weeks matter is still unresolved and I can not get through to a supervisor or manager to help
   112. negative  --     1 --  @united  very disappointed by the service starting from gate operator at BOS , who was  rude in stopping a carry-on bag...
   118. negative  --     1 --  @united what a pointless tweet. At least @AmericanAir asked me to follow them to try and resolve
   148. negative  --     1 --  @united how much longer United?? Been on the phone for over an hour to straighten out a star alliance upgrade!!! http://t.co/U1ViEuiDF5
   153. positive  --     4 --  It really is

In [13]:
sentences_list[10420]

'you re the reason this whole travel experience has been a nightmare '

In [18]:
df.head(1)

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone,clean_text
0,570018445695148033,negative,1.0,Customer Service Issue,0.656,United,NaN,tpensari,NaN,0,@united your customer service is terrible! Sto...,NaN,2015-02-23 16:32:42 -0800,NaN,NaN,your customer service is terrible _exclamation...


In [21]:
df["clean_text"]=sentences_list[2640:]

In [22]:
def print_df(df,start=0,stop=0):
    for i in range(start,stop):
        print(f"{i:4}. --{df.loc[i].airline_sentiment} --{df.loc[i].text} -- {df.loc[i].clean_text}")

In [23]:
print_df(df,1,10)

   1. --neutral --@SouthwestAir  I will dm you now -- i will do you now 
   2. --negative --@united congratulations united. Fail again. Sky at a gate with a plane full of people waiting on 2. Weather gets worse now we can't leave. -- congratulations united fail again sky at a gate with a plane full of people waiting on  weather gets worse now we can i leave 
   3. --positive --@united look at this beauty 😉 dc 10 united airlines 😉 http://t.co/MvYoizRPdE -- look at this beauty :winking_face: do  united airlines :winking_face: 
   4. --positive --@JetBlue Landing! As usual great flight wiyh a great crew. Hello sunny West Palm Beach ! #jetbluerocks -- landing _exclamation_ as usual great flight wiyh a great crew hello sunny west palm beach 
   5. --neutral --Thx! Stand by! “@united: @rajuchinthala I know it's frustrating and I do appreciate your patience while we try to get you on your way. ^JH” -- the _exclamation_ stand by _exclamation_ i know it i frustrating and i do appreciate your pa

In [24]:
all_sent=[]
for sentence in df.clean_text:
    sent_list=sentence.strip().split()
    out=[]
    for word in sent_list:
        out.append(word_dict[word]) 
    all_sent.append(out)

In [25]:
df["sentence_code"]=all_sent

In [26]:
table=list(df["sentence_code"]) # typo
lens=max([len(x) for x in table])
sentiment=list(df["airline_sentiment"])


In [27]:
def ylabel_convert(sentiment):
    ytrain=[]
    for sent in sentiment:
        if sent=="negative":
            o=[1,0]
        if sent=="neutral":
            o=[0.5,0.5]
        if sent=="positive":
            o=[0,1]
        ytrain.append(o)
    return(ytrain)

def ylabel_convert3(sentiment):
    ytrain=[]
    for sent in sentiment:
        if sent=="negative":
            o=[1,0,0]
        if sent=="neutral":
            o=[0,1,0]
        if sent=="positive":
            o=[0,0,1]
        ytrain.append(o)
    return(ytrain)

        

In [28]:
ytrain=ylabel_convert3(sentiment)

In [29]:
x0=[0 for _ in range(lens) ]
xtrain=[]
for sent in table:
    o1=list(x0[0:lens-len(sent)]+list(sent))
    xtrain.append(o1)

In [30]:
x_train=xtrain[0:-20]
y_train=ytrain[0:-20]
x_test=xtrain[-20:]
y_test=ytrain[-20:]

In [31]:


lstm_size=lens
max_input_length=lens
embedding_size=190 #(100: 73%)
n_words=len(table)
n_out=len(y_train[0])

In [32]:
lens
n_out

3

In [33]:

# Importáld a megfelelő rétegeket
from tensorflow.keras.layers import Input,Dense,Embedding,LSTM,TimeDistributed, Flatten, Bidirectional
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adadelta,Adam,SGD
from tensorflow.keras.losses import sparse_categorical_crossentropy,categorical_crossentropy, mean_squared_error
from tensorflow.keras import backend as K

K.clear_session
x= Input(shape=(max_input_length,))
embedded_x=Embedding(n_words+1,embedding_size, input_length=max_input_length-1, mask_zero=True)(x)
lstm_output= LSTM(lstm_size,return_sequences=True)(embedded_x)
lstm_output= LSTM(lstm_size,return_sequences=True)(lstm_output)


lstm_output=Flatten()(lstm_output)
#Dense_out= Dense(33, activation="softmax")(lstm_output)

predictions= Dense(n_out, activation="softmax")(lstm_output)
model=Model(inputs=x, outputs=predictions,)



In [34]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 59)]              0         
_________________________________________________________________
embedding (Embedding)        (None, 59, 190)           2280190   
_________________________________________________________________
lstm (LSTM)                  (None, 59, 59)            59000     
_________________________________________________________________
lstm_1 (LSTM)                (None, 59, 59)            28084     
_________________________________________________________________
flatten (Flatten)            (None, 3481)              0         
_________________________________________________________________
dense (Dense)                (None, 3)                 10446     
Total params: 2,377,720
Trainable params: 2,377,720
Non-trainable params: 0
___________________________________________________

In [35]:
x

<KerasTensor: shape=(None, 59) dtype=float32 (created by layer 'input_1')>

In [36]:
import numpy as np

In [37]:
#train_x = np.asarray(xtrain)
#train_y = np.asarray(ytrain)

In [38]:
# Loss 

loss = mean_squared_error #categorical_crossentropy # One-hot enkódolt kimenetünk van. Mit is használunk?

# Optimizer
optimizer = Adam() #Ízlés szerint...
 
# Compilation
#############

model.compile(optimizer=optimizer,loss=loss)

In [70]:
# Illesszük az adatra a modellt.
history=model.fit(x=xtrain,y=ytrain,validation_data=( x_test,y_test), epochs=6, batch_size=50,)


Epoch 1/6
240/240 [==============================] - 47s 196ms/step - loss: 0.0274 - val_loss: 0.0507
Epoch 2/6
240/240 [==============================] - 47s 196ms/step - loss: 0.0243 - val_loss: 3.7360e-04
Epoch 3/6
240/240 [==============================] - 47s 196ms/step - loss: 0.0218 - val_loss: 5.2806e-05
Epoch 4/6
240/240 [==============================] - 47s 196ms/step - loss: 0.0206 - val_loss: 1.1703e-05
Epoch 5/6
240/240 [==============================] - 47s 195ms/step - loss: 0.0189 - val_loss: 4.0360e-05
Epoch 6/6
240/240 [==============================] - 47s 197ms/step - loss: 0.0181 - val_loss: 5.4120e-05


In [40]:
pred=model.predict(x_train)

In [41]:
for i in range(len(pred)):
    print(f"{i}, {pred[i]}, {y_train[i]}")



A streamkimeneten csak az utolsó 5000 sor látható.
6980, [9.9999905e-01 9.4486228e-07 2.3139537e-08], [1, 0, 0]
6981, [0.19346827 0.6953614  0.11117035], [0, 1, 0]
6982, [8.656606e-03 9.907726e-01 5.708172e-04], [0, 1, 0]
6983, [0.96516746 0.03045646 0.00437609], [1, 0, 0]
6984, [9.9971813e-01 2.3963033e-04 4.2239830e-05], [1, 0, 0]
6985, [9.9998462e-01 2.5823001e-06 1.2715631e-05], [1, 0, 0]
6986, [9.9999905e-01 8.7646953e-07 1.0377702e-07], [1, 0, 0]
6987, [6.0340829e-02 9.3936527e-01 2.9395492e-04], [0, 1, 0]
6988, [9.9996459e-01 3.4995694e-05 4.1111903e-07], [1, 0, 0]
6989, [0.00669406 0.01500594 0.9783    ], [0, 0, 1]
6990, [9.9994040e-01 5.8095949e-05 1.5649523e-06], [1, 0, 0]
6991, [0.0365398  0.01388094 0.9495793 ], [0, 0, 1]
6992, [0.20821135 0.05421926 0.7375694 ], [0, 0, 1]
6993, [9.9997497e-01 2.4872857e-05 1.7338057e-07], [1, 0, 0]
6994, [0.0247979  0.9477096  0.02749253], [0, 1, 0]
6995, [5.939465e-04 8.326164e-05 9.993228e-01], [0, 0, 1]
6996, [9.9999774e-01 1.4699615e-0

### Model usage

In [42]:
test_df=pd.read_csv("TEST.csv")

In [43]:
print(len(test_df))

2640


In [44]:
test_df.head(130)

,tweet_id,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570031016800161792,1.0000,Lost Luggage,1.0000,United,NaN,tpensari,NaN,0,@united well someone should tell that to the e...,NaN,2015-02-23 17:22:39 -0800,NaN,NaN
1,569948646717812736,1.0000,NaN,NaN,American,NaN,Metalmikefisher,NaN,0,@AmericanAir Let's all have a extraordinary we...,NaN,2015-02-23 11:55:21 -0800,NaN,Eastern Time (US & Canada)
2,569469326547222528,0.6811,Late Flight,0.6811,United,NaN,jlongoria21,NaN,0,@united since I have an international connecti...,NaN,2015-02-22 04:10:42 -0800,"Rio Grande Valley, Texas",Central Time (US & Canada)
3,569512239209824256,1.0000,Customer Service Issue,0.6812,Delta,NaN,cshells4,NaN,0,@JetBlue woulda been nice of you to let us kno...,NaN,2015-02-22 07:01:13 -0800,Connecticut,NaN
4,567842370533867520,1.0000,Customer Service Issue,1.0000,US Airways,NaN,suzanneboles,NaN,0,@USAirways contd.. They put her on 7 pm flite ...,NaN,2015-02-17 16:25:45 -0800,"London, ON'",Eastern Time (US & Canada)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125,570285904809598977,1.0000,NaN,NaN,Virgin America,NaN,sjespers,NaN,0,@VirginAmerica Thanks!,NaN,2015-02-24 10:15:29 -0800,"San Francisco, CA",Pacific Time (US & Canada)
126,568650058780041216,1.0000,Customer Service Issue,1.0000,US Airways,NaN,lj_verde,NaN,0,@USAirways never flying #USAirways again. Horr...,NaN,2015-02-19 21:55:13 -0800,DC,Eastern Time (US & Canada)
127,568494200163074048,0.3796,NaN,0.0000,US Airways,NaN,J_0_Y,NaN,0,@USAirways I just think it's weird to have mil...,"[42.95119138, -87.90117857]",2015-02-19 11:35:54 -0800,"Washington, DC",NaN
128,570059530567962624,1.0000,Can't Tell,1.0000,United,NaN,_mhertz,NaN,1,@united @AmericanAir so that's it? It just end...,NaN,2015-02-23 19:15:57 -0800,NaN,Pacific Time (US & Canada)


In [45]:
test_df["clean_text"]=sentences_list[:2640]

In [46]:
def code_sentences(df):
    all_sent=[]
    for sentence in df.clean_text:
        sent_list=sentence.strip().split()
        out=[]
        for word in sent_list:
            out.append(word_dict[word]) 
        all_sent.append(out)
    df["sentence_code"]=all_sent
    return (df)

In [47]:
test_df=code_sentences(test_df)

In [48]:
def push_sentences(df,lens=33):
    table=list(df["sentence_code"]) # typo
    
    print(lens)
    x0=[0 for _ in range(lens) ]
    x=[]
    for sent in table:
        o1=list(x0[0:lens-len(sent)]+list(sent))
        x.append(o1)
    return(x)

In [49]:
test_df.head(1)

,tweet_id,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone,clean_text,sentence_code
0,570031016800161792,1.0,Lost Luggage,1.0,United,NaN,tpensari,NaN,0,@united well someone should tell that to the e...,NaN,2015-02-23 17:22:39 -0800,NaN,NaN,well someone should tell that to the employees...,"[9345, 7916, 7696, 8473, 8524, 8646, 8529, 302..."


In [50]:

xx=push_sentences(test_df,lens=59)

59


In [51]:
test_out=model.predict(xx)

In [52]:
test_out

array([[9.9999774e-01, 2.1444389e-06, 1.5615852e-07],
       [2.3646615e-02, 4.0182769e-03, 9.7233510e-01],
       [9.0507150e-01, 9.2718340e-02, 2.2101582e-03],
       ...,
       [9.9544936e-01, 2.3604184e-03, 2.1901971e-03],
       [9.9999964e-01, 1.7935312e-07, 1.2404178e-07],
       [5.3025001e-01, 4.6944499e-01, 3.0499743e-04]], dtype=float32)

In [53]:
def decode3(model_out):
    y=[]
    for mo in model_out:
        if mo[0]>mo[1] and mo[0]>mo[2]:
            o="negative"    
        if mo[1]>mo[0] and mo[1]>mo[2]:
            o="neutral"    
        if mo[2]>mo[0] and mo[2]>mo[1]:
            o="positive"    
        y.append(o)
    return(y)


  

In [54]:
def decode(model_out):
    y=[]
    for mo in model_out:
        dif=abs(mo[0]-mo[1])
        if dif < 0.2 :
            o="neutral"
        else:
            if mo[0]>mo[1] :
                o="negative"    
            else:
                o="positive"    
        y.append(o)
    return(y)



In [55]:
prediction=decode3(test_out)

In [56]:
def listing(status):
    counti=0
    for i in range(len(prediction)):
    
        predi=prediction[i]
        reason=test_df.loc[i].negativereason
        airline_sentiment=test_df.loc[i].airline_sentiment_gold
        if predi==status:
            counti+=1
            print(f"{i:4}. {counti:3}. =  {prediction[i]} -- {reason:15}--{airline_sentiment} -- {test_df.loc[i].text}")
            cond=[reason=="Flight Booking Problems", 
                    reason=="Bad Flight", 
                    reason=="Late Flight", 
                    reason=="Lost Luggage", 
                    reason=="Customer Service Issue",
                    reason=="Cancelled Flight",
                    reason=="Flight Attendant Complaints",
                    reason=="longlines",
                    reason=="Damaged Luggage"
                    ]
            if any(cond):
                prediction[i]="negative"


In [57]:
df.head(1)

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone,clean_text,sentence_code
0,570018445695148033,negative,1.0,Customer Service Issue,0.656,United,NaN,tpensari,NaN,0,@united your customer service is terrible! Sto...,NaN,2015-02-23 16:32:42 -0800,NaN,NaN,your customer service is terrible _exclamation...,"[9595, 2343, 7608, 4613, 8497, 364, 8137, 4474..."


In [58]:
def retweet_converter():
    counti=0
    for i in range(len(prediction)):
    
        predi=prediction[i]
        reason=test_df.loc[i].negativereason
        retweet=test_df.loc[i].retweet_count
        if retweet>0:
            prediction[i]="negative"

In [59]:
listing("neutral")
#retweet_converter()

   5.   1. =  neutral --             nan--nan -- @JetBlue saving my sanity. Leaving it behind for sunshine. #escape #FL #bliss #travel #InDenial #WhatFrozenPipes http://t.co/6TtzEJV3hY
   9.   2. =  neutral -- Can't Tell     --nan -- @JetBlue I thought being a mosaic member had 'perks' the best part is instead of checking the extra 5lbs I'm carrying it on 😒
  10.   3. =  neutral --             nan--nan -- @AmericanAir is there a probability that flight 1330 from DFW to DTW will be delayed or Cancelled Flighted?
  14.   4. =  neutral --             nan--nan -- @AmericanAir please dm me
  16.   5. =  neutral --             nan--nan -- @SouthwestAir is there a way to know who checked my bag on the curb? She was awesome!!! And want to be sure she gets a high five!
  17.   6. =  neutral -- Customer Service Issue--nan -- @united Your "Loyalty Team" basically flipped me off via phone, but thanks. Maybe Google "loyalty" and get back to me? ^LOL
  23.   7. =  neutral -- Flight Attendant Complai

In [60]:
test_df["predicted"]=prediction

In [61]:
prediction_df=pd.DataFrame(prediction)

In [62]:
prediction_df.columns=["Sentiment"]

In [63]:
prediction_df.head()

,Sentiment
0,negative
1,positive
2,negative
3,negative
4,negative


In [64]:
prediction_df.index.name="Index"

In [65]:
from datetime  import datetime

In [66]:
a=datetime.now().strftime("%Y%m%d_%H%M%S")

In [67]:
print(a)

20211010_084101


In [68]:
prediction_df.to_csv("submission_"+a+".csv")

In [69]:
predcsv=test_df.loc[:,["text","predicted"]]

predcsv.to_csv("mini.csv")